In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from functools import reduce
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 150)
def percentchange(x, y):
    try:
        return ((x - y)*100/y)
    except ZeroDivisionError:
        return 0
def realchange(x, y):
    return x-y
def percent(x, y):
    return (x/y)*100

## This notebook outlines the high level industry data formatting for the Comprehensive Plans with place level primary geographies. There is another document for the information that is pulled for all relevant geographies.

The Woods & Poole data that is used more at the county level is only used at the place level for total employment projections. This is for consistency. This is a land use model ultimately, so JobsEQ employment is selected to model place level employment for place of work. This data is derived from the QCEW and other inputs so is selected as a better option - both are modeled so we use the one modeled directly from UI claims rather than land use at this granular of a geography.

To retrieve data:
+ Log into JobsEQ and navigate to the "Data Explorer" function  
+ go to "Draft Mode"
+ Select the "Industry" dataset, quarterly  
+ Select the following dimensions: Region, Industry  
+ Select the following values: "Employment (4Q Avg)", "Avg Ann Wages per Worker", "Location Quotient (4Q Avg)", Forecast Growth Demand (3 Years), Forecast Growth Rate (3 Years)"
+ Under "Region" select "USA", "Tennessee", then "GNRC Region" and sub all "Counties" and "Census Places"   
+ Under "Industry" expand "Total-All Industries" and select 2 digit NAICS codes, leaving Total as well

Once it is completed, open and delete the top row that says "Industry Data", and delete the FIPS column. Rename the Region column to NAME, delete the data notes at the bottom.  "JobsEQIndustryEmployment_Place.csv"

In [73]:
data = pd.read_csv('../../Data Downloads/JobsEQIndustryEmployment_allgeostest.csv')

In [74]:
data.head()

,NAME,Total - All Industries,Total - All Industries.1,Total - All Industries.2,Total - All Industries.3,Total - All Industries.4,Construction (23),Construction (23).1,Construction (23).2,Construction (23).3,Construction (23).4,Manufacturing (31),Manufacturing (31).1,Manufacturing (31).2,Manufacturing (31).3,Manufacturing (31).4,Information (51),Information (51).1,Information (51).2,Information (51).3,Information (51).4,Public Administration (92),Public Administration (92).1,Public Administration (92).2,Public Administration (92).3,Public Administration (92).4,Other/Unclassified,Other/Unclassified.1,Other/Unclassified.2,Other/Unclassified.3,Other/Unclassified.4,Leisure & Hospitality,Leisure & Hospitality.1,Leisure & Hospitality.2,Leisure & Hospitality.3,Leisure & Hospitality.4,Education & Health Services,Education & Health Services.1,Education & Health Services.2,Education & Health Services.3,Education & Health Services.4,Professional & Business Services,Professional & Business Services.1,Professional & Business Services.2,Professional & Business Services.3,Professional & Business Services.4,Financial Activities,Financial Activities.1,Financial Activities.2,Financial Activities.3,Financial Activities.4,"Trade, Transportation, & Utilities","Trade, Transportation, & Utilities.1","Trade, Transportation, & Utilities.2","Trade, Transportation, & Utilities.3","Trade, Transportation, & Utilities.4",Natural Resources & Mining,Natural Resources & Mining.1,Natural Resources & Mining.2,Natural Resources & Mining.3,Natural Resources & Mining.4
0,NAME,Empl (4Q Mov Avg),Avg Ann Wages per Worker,LQ (4Q Mov Avg),3 Year Forecast Growth Demand,3 Year Forecast Growth Rate,Empl (4Q Mov Avg),Avg Ann Wages per Worker,LQ (4Q Mov Avg),3 Year Forecast Growth Demand,3 Year Forecast Growth Rate,Empl (4Q Mov Avg),Avg Ann Wages per Worker,LQ (4Q Mov Avg),3 Year Forecast Growth Demand,3 Year Forecast Growth Rate,Empl (4Q Mov Avg),Avg Ann Wages per Worker,LQ (4Q Mov Avg),3 Year Forecast Growth Demand,3 Year Forecast Growth Rate,Empl (4Q Mov Avg),Avg Ann Wages per Worker,LQ (4Q Mov Avg),3 Year Forecast Growth Demand,3 Year Forecast Growth Rate,Empl (4Q Mov Avg),Avg Ann Wages per Worker,LQ (4Q Mov Avg),3 Year Forecast Growth Demand,3 Year Forecast Growth Rate,Empl (4Q Mov Avg),Avg Ann Wages per Worker,LQ (4Q Mov Avg),3 Year Forecast Growth Demand,3 Year Forecast Growth Rate,Empl (4Q Mov Avg),Avg Ann Wages per Worker,LQ (4Q Mov Avg),3 Year Forecast Growth Demand,3 Year Forecast Growth Rate,Empl (4Q Mov Avg),Avg Ann Wages per Worker,LQ (4Q Mov Avg),3 Year Forecast Growth Demand,3 Year Forecast Growth Rate,Empl (4Q Mov Avg),Avg Ann Wages per Worker,LQ (4Q Mov Avg),3 Year Forecast Growth Demand,3 Year Forecast Growth Rate,Empl (4Q Mov Avg),Avg Ann Wages per Worker,LQ (4Q Mov Avg),3 Year Forecast Growth Demand,3 Year Forecast Growth Rate,Empl (4Q Mov Avg),Avg Ann Wages per Worker,LQ (4Q Mov Avg),3 Year Forecast Growth Demand,3 Year Forecast Growth Rate
1,USA,160228873,68132,1,2613334,0.016,9518560,65968,1,78547,0.008,12958908,79193,1,-49159,-0.004,3260122,142955,1,97593,0.03,7396552,76160,1,-31218,-0.004,7166677,41300,1,180150,0.025,16620497,30890,1,738170,0.044,35724749,60780,1,1014551,0.028,24578092,90645,1,466469,0.019,9306460,115801,1,83944,0.009,31000555,56820,1,16836,0.001,2697701,59074,1,45554,0.017
2,Tennessee,3393472,60032,1,85719,0.025,191297,63345,0.95,3883,0.02,363799,68078,1.33,2010,0.006,55201,92714,0.8,2658,0.048,142672,58916,0.91,552,0.004,156280,35456,1.03,5274,0.034,357050,30253,1.01,19427,0.054,677698,59021,0.9,21832,0.032,501125,72547,0.96,15003,0.03,177617,94388,0.9,4484,0.025,729632,57664,1.11,9524,0.013,41101,26876,0.72,228,0.006
3,GNRC Region,1179322,66465,1,58577,0.05,70124,71442,1,3010,0.043,90017,70978,0.94,3209,0.036,27988,98341,1.17,1718,0.061,49728,62239,0.91,1181,0.024,55311,39007,1.05,3315,0.06,135684,37914,1.11,10183,0.075,230540,65627,0.88,12460,0.054,199036,84731,1.1,9910,0.05,72834,109153,1.06,3325,0.046,240984,56109,1.06,9796,0.

In [75]:
data = data.set_index('NAME').transpose()
data.head()

NAME,NAME,USA,Tennessee,GNRC Region,"Adams city, TN","Ashland City town, TN","Belle Meade city, TN","Berry Hill city, TN","Bethpage CDP, TN","Bransford CDP, TN","Brentwood city, TN","Burns town, TN","Castalian Springs CDP, TN","Cedar Hill city, TN","Charlotte town, TN","Christiana CDP, TN","Clarksville city, TN","Columbia city, TN","Coopertown town, TN","Cottontown CDP, TN","Cross Plains city, TN","Cumberland City town, TN","Dickson city, TN","Dover city, TN","Eagleville city, TN","Erin city, TN","Fairfield CDP, TN","Fairview city, TN","Forest Hills city, TN","Franklin city, TN","Gallatin city, TN","Goodlettsville city, TN","Graball CDP, TN","Greenbrier town, TN","Green Hill CDP, TN","Hartsville/Trousdale County, TN","Hendersonville city, TN","Kingston Springs town, TN","La Vergne city, TN","Lebanon city, TN","McEwen city, TN","Millersville city, TN","Mitchellville city, TN","Mount Juliet city, TN","Mount Pleasant city, TN","Murfreesboro city, TN","Nashville-Davidson metropolitan government (balance), TN","New Deal CDP, TN","New Johnsonville city, TN","Nolensville town, TN","Oak Grove CDP, TN","Oak Hill city, TN","Orlinda city, TN","Pegram town, TN","Pleasant View city, TN","Portland city, TN","Ridgetop city, TN","Rockvale CDP, TN","Rural Hill CDP, TN","Shackle Island CDP, TN","Slayden town, TN","Smyrna town, TN","Springfield city, TN","Spring Hill city, TN","Tennessee Ridge town, TN","Thompson's Station town, TN","Vanleer town, TN","Walnut Grove CDP, TN","Walterhill CDP, TN","Watertown city, TN","Waverly city, TN","Westmoreland town, TN","White Bluff town, TN","White House city, TN","Cheatham County, Tennessee","Davidson County, Tennessee","Dickson County, Tennessee","Houston County, Tennessee","Humphreys County, Tennessee","Maury County, Tennessee","Montgomery County, Tennessee","Robertson County, Tennessee","Rutherford County, Tennessee","Stewart County, Tennessee","Sumner County, Tennessee","Trousdale County, Tennessee","Williamson County, Tennessee","Wilson County, Tennessee"
Total - All Industries,Empl (4Q Mov Avg),160228873,3393472,1179322,67,5004,1319,18306,66,7,57368,194,45,64,1029,378,54378,22584,1700,20,295,820,12897,995,481,902,1,1886,897,82917,23352,14016,2,1249,680,2302,25332,903,18354,34322,509,1397,28,18180,2159,76307,524693,58,649,2768,61,1223,444,298,1602,7850,248,126,172,456,17,41763,11300,10403,231,1751,53,49,204,560,2151,899,935,5011,10901,558576,20319,1784,6502,41515,64976,25726,149974,3331,66405,2302,160642,66367
Total - All Industries.1,Avg Ann Wages per Worker,68132,60032,66465,46817,52603,72434,69947,48847,54854,82660,45854,60087,47178,46756,50247,45945,54656,46107,48987,45052,50402,47137,46549,48871,37204,47810,74387,71847,81474,52571,71855,43961,47234,52347,45075,52982,50383,56281,51402,48185,51443,53822,51205,53428,55389,74563,51278,52383,75921,54043,69438,43639,48202,52366,53244,44698,51234,54640,47128,54430,56023,47991,62870,36002,69534,48013,56881,54652,48790,50295,52132,47100,47779,50865,74369,46358,35120,51422,53788,45879,47383,55830,51676,52670,44897,80860,50946
Total - All Industries.2,LQ (4Q Mov Avg),1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
Total - All Industries.3,3 Year Forecast Growth Demand,2613334,85719,58577,2,81,60,333,3,0,4543,5,1,2,24,21,2744,1445,54,1,6,10,432,15,28,0,0,129,41,6537,1208,468,0,33,62,211,1478,21,1222,3139,3,62,2,1730,120,4895,17673,3,3,209,3,44,13,6,47,388,8,6,15,20,0,2614,324,724,-1,112,1,2,11,43,22,40,11,194,222,18514,632,-2,45,2628,3162,815,9519,25,3586,211,12570,6104
Total - All Industries.4,3 Year Forecast Growth Rate,0.016,0.025,0.05,0.037,0.016,0.046,0.018,0.039,0.045,0.079,0.028,0.032,0.031,0.023,0.056,0.05,0.064,0.032,0.059,0.021,0.012,0.033,0.015,0.057,0,0.042,0.068,0.045,0.079,0.052,0.033,0.044,0.027,0.091,0.092,0.058,0.024,0.067,0.091,0.006,0.045,0.054,0.095,0.056,0.064,0.034,0.052,0.004,0.075,0.05,0.036,0.029,0.021,0.03,0.

In [39]:
data = data.rename(columns = {'NAME': 'Value'})

In [40]:
data = data.reset_index(drop = False)

In [41]:
data.head()

NAME,index,Value,USA,Tennessee,GNRC Region,"Adams city, TN","Ashland City town, TN","Belle Meade city, TN","Berry Hill city, TN","Bethpage CDP, TN","Bransford CDP, TN","Brentwood city, TN","Burns town, TN","Castalian Springs CDP, TN","Cedar Hill city, TN","Charlotte town, TN","Christiana CDP, TN","Clarksville city, TN","Columbia city, TN","Coopertown town, TN","Cottontown CDP, TN","Cross Plains city, TN","Cumberland City town, TN","Dickson city, TN","Dover city, TN","Eagleville city, TN","Erin city, TN","Fairfield CDP, TN","Fairview city, TN","Forest Hills city, TN","Franklin city, TN","Gallatin city, TN","Goodlettsville city, TN","Graball CDP, TN","Greenbrier town, TN","Green Hill CDP, TN","Hartsville/Trousdale County, TN","Hendersonville city, TN","Kingston Springs town, TN","La Vergne city, TN","Lebanon city, TN","McEwen city, TN","Millersville city, TN","Mitchellville city, TN","Mount Juliet city, TN","Mount Pleasant city, TN","Murfreesboro city, TN","Nashville-Davidson metropolitan government (balance), TN","New Deal CDP, TN","New Johnsonville city, TN","Nolensville town, TN","Oak Grove CDP, TN","Oak Hill city, TN","Orlinda city, TN","Pegram town, TN","Pleasant View city, TN","Portland city, TN","Ridgetop city, TN","Rockvale CDP, TN","Rural Hill CDP, TN","Shackle Island CDP, TN","Slayden town, TN","Smyrna town, TN","Springfield city, TN","Spring Hill city, TN","Tennessee Ridge town, TN","Thompson's Station town, TN","Vanleer town, TN","Walnut Grove CDP, TN","Walterhill CDP, TN","Watertown city, TN","Waverly city, TN","Westmoreland town, TN","White Bluff town, TN","White House city, TN","Cheatham County, Tennessee","Davidson County, Tennessee","Dickson County, Tennessee","Houston County, Tennessee","Humphreys County, Tennessee","Maury County, Tennessee","Montgomery County, Tennessee","Robertson County, Tennessee","Rutherford County, Tennessee","Stewart County, Tennessee","Sumner County, Tennessee","Trousdale County, Tennessee","Williamson County, Tennessee","Wilson County, Tennessee"
0,Total - All Industries,Empl (4Q Mov Avg),160228873,3393472,1179322,67,5004,1319,18306,66,7,57368,194,45,64,1029,378,54378,22584,1700,20,295,820,12897,995,481,902,1,1886,897,82917,23352,14016,2,1249,680,2302,25332,903,18354,34322,509,1397,28,18180,2159,76307,524693,58,649,2768,61,1223,444,298,1602,7850,248,126,172,456,17,41763,11300,10403,231,1751,53,49,204,560,2151,899,935,5011,10901,558576,20319,1784,6502,41515,64976,25726,149974,3331,66405,2302,160642,66367
1,Total - All Industries.1,Avg Ann Wages per Worker,68132,60032,66465,46817,52603,72434,69947,48847,54854,82660,45854,60087,47178,46756,50247,45945,54656,46107,48987,45052,50402,47137,46549,48871,37204,47810,74387,71847,81474,52571,71855,43961,47234,52347,45075,52982,50383,56281,51402,48185,51443,53822,51205,53428,55389,74563,51278,52383,75921,54043,69438,43639,48202,52366,53244,44698,51234,54640,47128,54430,56023,47991,62870,36002,69534,48013,56881,54652,48790,50295,52132,47100,47779,50865,74369,46358,35120,51422,53788,45879,47383,55830,51676,52670,44897,80860,50946
2,Total - All Industries.2,LQ (4Q Mov Avg),1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
3,Total - All Industries.3,3 Year Forecast Growth Demand,2613334,85719,58577,2,81,60,333,3,0,4543,5,1,2,24,21,2744,1445,54,1,6,10,432,15,28,0,0,129,41,6537,1208,468,0,33,62,211,1478,21,1222,3139,3,62,2,1730,120,4895,17673,3,3,209,3,44,13,6,47,388,8,6,15,20,0,2614,324,724,-1,112,1,2,11,43,22,40,11,194,222,18514,632,-2,45,2628,3162,815,9519,25,3586,211,12570,6104
4,Total - All Industries.4,3 Year Forecast Growth Rate,0.016,0.025,0.05,0.037,0.016,0.046,0.018,0.039,0.045,0.079,0.028,0.032,0.031,0.023,0.056,0.05,0.064,0.032,0.059,0.021,0.012,0.033,0.015,0.057,0,0.042,0.068,0.045,0.079,0.052,0.033,0.044,0.027,0.091,0.092,0.058,0.024,0.067,0.091,0.006,0.045,0.054,0.095,0.056,0.064,0.034,0.052,0.004,0.075,0.05,0.036,0.

In [42]:
data['index'] = data['index'].str.strip()
data['Value'] = data['Value'].str.strip()

In [43]:
inds = ['Construction (23)', 'Manufacturing (31)', 'Information (51)', 'Public Administration (92)', 'Other/Unclassified', 'Leisure & Hospitality', 
        'Education & Health Services', 'Professional & Business Services', 'Financial Activities', 'Trade, Transportation, & Utilities', 'Natural Resources & Mining']

In [44]:
test = data.set_index(['index', 'Value']).transpose()
test.head()

index,Total - All Industries,Total - All Industries.1,Total - All Industries.2,Total - All Industries.3,Total - All Industries.4,Construction (23),Construction (23).1,Construction (23).2,Construction (23).3,Construction (23).4,Manufacturing (31),Manufacturing (31).1,Manufacturing (31).2,Manufacturing (31).3,Manufacturing (31).4,Information (51),Information (51).1,Information (51).2,Information (51).3,Information (51).4,Public Administration (92),Public Administration (92).1,Public Administration (92).2,Public Administration (92).3,Public Administration (92).4,Other/Unclassified,Other/Unclassified.1,Other/Unclassified.2,Other/Unclassified.3,Other/Unclassified.4,Leisure & Hospitality,Leisure & Hospitality.1,Leisure & Hospitality.2,Leisure & Hospitality.3,Leisure & Hospitality.4,Education & Health Services,Education & Health Services.1,Education & Health Services.2,Education & Health Services.3,Education & Health Services.4,Professional & Business Services,Professional & Business Services.1,Professional & Business Services.2,Professional & Business Services.3,Professional & Business Services.4,Financial Activities,Financial Activities.1,Financial Activities.2,Financial Activities.3,Financial Activities.4,"Trade, Transportation, & Utilities","Trade, Transportation, & Utilities.1","Trade, Transportation, & Utilities.2","Trade, Transportation, & Utilities.3","Trade, Transportation, & Utilities.4",Natural Resources & Mining,Natural Resources & Mining.1,Natural Resources & Mining.2,Natural Resources & Mining.3,Natural Resources & Mining.4
Value,Empl (4Q Mov Avg),Avg Ann Wages per Worker,LQ (4Q Mov Avg),3 Year Forecast Growth Demand,3 Year Forecast Growth Rate,Empl (4Q Mov Avg),Avg Ann Wages per Worker,LQ (4Q Mov Avg),3 Year Forecast Growth Demand,3 Year Forecast Growth Rate,Empl (4Q Mov Avg),Avg Ann Wages per Worker,LQ (4Q Mov Avg),3 Year Forecast Growth Demand,3 Year Forecast Growth Rate,Empl (4Q Mov Avg),Avg Ann Wages per Worker,LQ (4Q Mov Avg),3 Year Forecast Growth Demand,3 Year Forecast Growth Rate,Empl (4Q Mov Avg),Avg Ann Wages per Worker,LQ (4Q Mov Avg),3 Year Forecast Growth Demand,3 Year Forecast Growth Rate,Empl (4Q Mov Avg),Avg Ann Wages per Worker,LQ (4Q Mov Avg),3 Year Forecast Growth Demand,3 Year Forecast Growth Rate,Empl (4Q Mov Avg),Avg Ann Wages per Worker,LQ (4Q Mov Avg),3 Year Forecast Growth Demand,3 Year Forecast Growth Rate,Empl (4Q Mov Avg),Avg Ann Wages per Worker,LQ (4Q Mov Avg),3 Year Forecast Growth Demand,3 Year Forecast Growth Rate,Empl (4Q Mov Avg),Avg Ann Wages per Worker,LQ (4Q Mov Avg),3 Year Forecast Growth Demand,3 Year Forecast Growth Rate,Empl (4Q Mov Avg),Avg Ann Wages per Worker,LQ (4Q Mov Avg),3 Year Forecast Growth Demand,3 Year Forecast Growth Rate,Empl (4Q Mov Avg),Avg Ann Wages per Worker,LQ (4Q Mov Avg),3 Year Forecast Growth Demand,3 Year Forecast Growth Rate,Empl (4Q Mov Avg),Avg Ann Wages per Worker,LQ (4Q Mov Avg),3 Year Forecast Growth Demand,3 Year Forecast Growth Rate
NAME,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
USA,160228873,68132,1,2613334,0.016,9518560,65968,1,78547,0.008,12958908,79193,1,-49159,-0.004,3260122,142955,1,97593,0.03,7396552,76160,1,-31218,-0.004,7166677,41300,1,180150,0.025,16620497,30890,1,738170,0.044,35724749,60780,1,1014551,0.028,24578092,90645,1,466469,0.019,9306460,115801,1,83944,0.009,31000555,56820,1,16836,0.001,2697701,59074,1,45554,0.017
Tennessee,3393472,60032,1,85719,0.025,191297,63345,0.95,3883,0.02,363799,68078,1.33,2010,0.006,55201,92714,0.8,2658,0.048,142672,58916,0.91,552,0.004,156280,35456,1.03,5274,0.034,357050,30253,1.01,19427,0.054,677698,59021,0.9,21832,0.032,501125,72547,0.96,15003,0.03,177617,94388,0.9,4484,0.025,729632,57664,1.11,9524,0.013,41101,26876,0.72,228,0.006
GNRC Region,1179322,66465,1,58577,0.05,70124,71442,1,3010,0.043,90017,70978,0.94,3209,0.036,27988,98341,1.17,1718,0.061,49728,62239,0.91,1181,0.024,55311,39007,1.05,3315,0.06,135684,37914,1.11,10183,0.075,230540,65627,0.88,12460,0.054,199036,84731,1.1,9910,0

In [45]:
test = test.fillna(0)

In [46]:
test.head(2)

index,Total - All Industries,Total - All Industries.1,Total - All Industries.2,Total - All Industries.3,Total - All Industries.4,Construction (23),Construction (23).1,Construction (23).2,Construction (23).3,Construction (23).4,Manufacturing (31),Manufacturing (31).1,Manufacturing (31).2,Manufacturing (31).3,Manufacturing (31).4,Information (51),Information (51).1,Information (51).2,Information (51).3,Information (51).4,Public Administration (92),Public Administration (92).1,Public Administration (92).2,Public Administration (92).3,Public Administration (92).4,Other/Unclassified,Other/Unclassified.1,Other/Unclassified.2,Other/Unclassified.3,Other/Unclassified.4,Leisure & Hospitality,Leisure & Hospitality.1,Leisure & Hospitality.2,Leisure & Hospitality.3,Leisure & Hospitality.4,Education & Health Services,Education & Health Services.1,Education & Health Services.2,Education & Health Services.3,Education & Health Services.4,Professional & Business Services,Professional & Business Services.1,Professional & Business Services.2,Professional & Business Services.3,Professional & Business Services.4,Financial Activities,Financial Activities.1,Financial Activities.2,Financial Activities.3,Financial Activities.4,"Trade, Transportation, & Utilities","Trade, Transportation, & Utilities.1","Trade, Transportation, & Utilities.2","Trade, Transportation, & Utilities.3","Trade, Transportation, & Utilities.4",Natural Resources & Mining,Natural Resources & Mining.1,Natural Resources & Mining.2,Natural Resources & Mining.3,Natural Resources & Mining.4
Value,Empl (4Q Mov Avg),Avg Ann Wages per Worker,LQ (4Q Mov Avg),3 Year Forecast Growth Demand,3 Year Forecast Growth Rate,Empl (4Q Mov Avg),Avg Ann Wages per Worker,LQ (4Q Mov Avg),3 Year Forecast Growth Demand,3 Year Forecast Growth Rate,Empl (4Q Mov Avg),Avg Ann Wages per Worker,LQ (4Q Mov Avg),3 Year Forecast Growth Demand,3 Year Forecast Growth Rate,Empl (4Q Mov Avg),Avg Ann Wages per Worker,LQ (4Q Mov Avg),3 Year Forecast Growth Demand,3 Year Forecast Growth Rate,Empl (4Q Mov Avg),Avg Ann Wages per Worker,LQ (4Q Mov Avg),3 Year Forecast Growth Demand,3 Year Forecast Growth Rate,Empl (4Q Mov Avg),Avg Ann Wages per Worker,LQ (4Q Mov Avg),3 Year Forecast Growth Demand,3 Year Forecast Growth Rate,Empl (4Q Mov Avg),Avg Ann Wages per Worker,LQ (4Q Mov Avg),3 Year Forecast Growth Demand,3 Year Forecast Growth Rate,Empl (4Q Mov Avg),Avg Ann Wages per Worker,LQ (4Q Mov Avg),3 Year Forecast Growth Demand,3 Year Forecast Growth Rate,Empl (4Q Mov Avg),Avg Ann Wages per Worker,LQ (4Q Mov Avg),3 Year Forecast Growth Demand,3 Year Forecast Growth Rate,Empl (4Q Mov Avg),Avg Ann Wages per Worker,LQ (4Q Mov Avg),3 Year Forecast Growth Demand,3 Year Forecast Growth Rate,Empl (4Q Mov Avg),Avg Ann Wages per Worker,LQ (4Q Mov Avg),3 Year Forecast Growth Demand,3 Year Forecast Growth Rate,Empl (4Q Mov Avg),Avg Ann Wages per Worker,LQ (4Q Mov Avg),3 Year Forecast Growth Demand,3 Year Forecast Growth Rate
NAME,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
USA,160228873,68132,1,2613334,0.016,9518560,65968,1,78547,0.008,12958908,79193,1,-49159,-0.004,3260122,142955,1,97593,0.03,7396552,76160,1,-31218,-0.004,7166677,41300,1,180150,0.025,16620497,30890,1,738170,0.044,35724749,60780,1,1014551,0.028,24578092,90645,1,466469,0.019,9306460,115801,1,83944,0.009,31000555,56820,1,16836,0.001,2697701,59074,1,45554,0.017
Tennessee,3393472,60032,1,85719,0.025,191297,63345,0.95,3883,0.02,363799,68078,1.33,2010,0.006,55201,92714,0.8,2658,0.048,142672,58916,0.91,552,0.004,156280,35456,1.03,5274,0.034,357050,30253,1.01,19427,0.054,677698,59021,0.9,21832,0.032,501125,72547,0.96,15003,0.03,177617,94388,0.9,4484,0.025,729632,57664,1.11,9524,0.013,41101,26876,0.72,228,0.006


In [47]:
cols = test.columns
test[cols] = test[cols].astype(float)

In [48]:
# thelist = [test[('Construction (23)', 'Empl (4Q Mov Avg)')], test[('Manufacturing (31)', 'Empl (4Q Mov Avg)')], 
#                test[('Information (51)', 'Empl (4Q Mov Avg)')], test[('Public Administration (92)', 'Empl (4Q Mov Avg)')],
#                test[('Other/Unclassified', 'Empl (4Q Mov Avg)')], test[('Leisure & Hospitality', 'Empl (4Q Mov Avg)')], 
#                test[('Education & Health Services', 'Empl (4Q Mov Avg)')], test[('Professional & Business Services', 'Empl (4Q Mov Avg)')], 
#                test[('Financial Activities', 'Empl (4Q Mov Avg)')], test[('Trade, Transportation, & Utilities', 'Empl (4Q Mov Avg)')], 
#                test[('Natural Resources & Mining', 'Empl (4Q Mov Avg)')] ]
# test[('Total - All Industries', 'Empl (4Q Mov Avg)')] = sum(thelist)

In [49]:
for ind in inds:
    test[('{}'.format(ind), 'Percent of Total')] = percent(test[('{}'.format(ind), 'Empl (4Q Mov Avg)')], test[('Total - All Industries', 'Empl (4Q Mov Avg)')])

In [50]:
test.head()

index,Total - All Industries,Total - All Industries.1,Total - All Industries.2,Total - All Industries.3,Total - All Industries.4,Construction (23),Construction (23).1,Construction (23).2,Construction (23).3,Construction (23).4,Manufacturing (31),Manufacturing (31).1,Manufacturing (31).2,Manufacturing (31).3,Manufacturing (31).4,Information (51),Information (51).1,Information (51).2,Information (51).3,Information (51).4,Public Administration (92),Public Administration (92).1,Public Administration (92).2,Public Administration (92).3,Public Administration (92).4,Other/Unclassified,Other/Unclassified.1,Other/Unclassified.2,Other/Unclassified.3,Other/Unclassified.4,Leisure & Hospitality,Leisure & Hospitality.1,Leisure & Hospitality.2,Leisure & Hospitality.3,Leisure & Hospitality.4,Education & Health Services,Education & Health Services.1,Education & Health Services.2,Education & Health Services.3,Education & Health Services.4,Professional & Business Services,Professional & Business Services.1,Professional & Business Services.2,Professional & Business Services.3,Professional & Business Services.4,Financial Activities,Financial Activities.1,Financial Activities.2,Financial Activities.3,Financial Activities.4,"Trade, Transportation, & Utilities","Trade, Transportation, & Utilities.1","Trade, Transportation, & Utilities.2","Trade, Transportation, & Utilities.3","Trade, Transportation, & Utilities.4",Natural Resources & Mining,Natural Resources & Mining.1,Natural Resources & Mining.2,Natural Resources & Mining.3,Natural Resources & Mining.4,Construction (23),Manufacturing (31),Information (51),Public Administration (92),Other/Unclassified,Leisure & Hospitality,Education & Health Services,Professional & Business Services,Financial Activities,"Trade, Transportation, & Utilities",Natural Resources & Mining
Value,Empl (4Q Mov Avg),Avg Ann Wages per Worker,LQ (4Q Mov Avg),3 Year Forecast Growth Demand,3 Year Forecast Growth Rate,Empl (4Q Mov Avg),Avg Ann Wages per Worker,LQ (4Q Mov Avg),3 Year Forecast Growth Demand,3 Year Forecast Growth Rate,Empl (4Q Mov Avg),Avg Ann Wages per Worker,LQ (4Q Mov Avg),3 Year Forecast Growth Demand,3 Year Forecast Growth Rate,Empl (4Q Mov Avg),Avg Ann Wages per Worker,LQ (4Q Mov Avg),3 Year Forecast Growth Demand,3 Year Forecast Growth Rate,Empl (4Q Mov Avg),Avg Ann Wages per Worker,LQ (4Q Mov Avg),3 Year Forecast Growth Demand,3 Year Forecast Growth Rate,Empl (4Q Mov Avg),Avg Ann Wages per Worker,LQ (4Q Mov Avg),3 Year Forecast Growth Demand,3 Year Forecast Growth Rate,Empl (4Q Mov Avg),Avg Ann Wages per Worker,LQ (4Q Mov Avg),3 Year Forecast Growth Demand,3 Year Forecast Growth Rate,Empl (4Q Mov Avg),Avg Ann Wages per Worker,LQ (4Q Mov Avg),3 Year Forecast Growth Demand,3 Year Forecast Growth Rate,Empl (4Q Mov Avg),Avg Ann Wages per Worker,LQ (4Q Mov Avg),3 Year Forecast Growth Demand,3 Year Forecast Growth Rate,Empl (4Q Mov Avg),Avg Ann Wages per Worker,LQ (4Q Mov Avg),3 Year Forecast Growth Demand,3 Year Forecast Growth Rate,Empl (4Q Mov Avg),Avg Ann Wages per Worker,LQ (4Q Mov Avg),3 Year Forecast Growth Demand,3 Year Forecast Growth Rate,Empl (4Q Mov Avg),Avg Ann Wages per Worker,LQ (4Q Mov Avg),3 Year Forecast Growth Demand,3 Year Forecast Growth Rate,Percent of Total,Percent of Total,Percent of Total,Percent of Total,Percent of Total,Percent of Total,Percent of Total,Percent of Total,Percent of Total,Percent of Total,Percent of Total
NAME,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
USA,160228873.0,68132.0,1.0,2613334.0,0.016,9518560.0,65968.0,1.00,78547.0,0.008,12958908.0,79193.0,1.00,-49159.0,-0.004,3260122.0,142955.0,1.00,97593.0,0.030,7396552.0,76160.0,1.00,-31218.0,-0.004,7166677.0,41300.0,1.00,180150.0,0.025,16620497.0,30890.0,1.00,738170.0,0.044,35724749.0,60780.0,1.00,1014551.0,0.028,24578092.0,90645.0,1.00,466469.0,0.019,9306460.0,115801.0,1.00,83944.0,0.009,31000555.0,56820.0,1.00,16836.0,0.001,2697701.0,59074.0,1.00,45554.0,0.017,5.940602,8.087748,2.034666,4.616242

In [51]:
test = test.transpose()

In [52]:
test = test.reset_index(drop = False)

In [53]:
boop = test['index'].str.split(pat = ".", expand = True)
test['Industry'] = boop[0]

In [54]:
data = test

In [55]:
data = data.drop(columns = 'index')

In [56]:
cols = data.columns
data = data.melt(var_name = 'NAME', id_vars = ['Value', 'Industry'], value_vars = cols, value_name = 'IDK')
data.head()

,Value,Industry,NAME,IDK
0,Empl (4Q Mov Avg),Total - All Industries,USA,1.602289e+08
1,Avg Ann Wages per Worker,Total - All Industries,USA,6.813200e+04
2,LQ (4Q Mov Avg),Total - All Industries,USA,1.000000e+00
3,3 Year Forecast Growth Demand,Total - All Industries,USA,2.613334e+06
4,3 Year Forecast Growth Rate,Total - All Industries,USA,1.600000e-02


In [57]:
data = data.pivot(index = ['Industry', 'NAME'], columns = 'Value')['IDK']

In [58]:
data = data.reset_index(drop = False)

In [59]:
data.head()

Value,Industry,NAME,3 Year Forecast Growth Demand,3 Year Forecast Growth Rate,Avg Ann Wages per Worker,Empl (4Q Mov Avg),LQ (4Q Mov Avg),Percent of Total
0,Construction (23),"Adams city, TN",0.0,0.032,0.0,0.0,0.08,0.000000
1,Construction (23),"Ashland City town, TN",4.0,0.025,56502.0,182.0,0.61,3.637090
2,Construction (23),"Belle Meade city, TN",1.0,0.023,75583.0,27.0,0.34,2.047005
3,Construction (23),"Berry Hill city, TN",23.0,0.023,75980.0,1005.0,0.92,5.490003
4,Construction (23),"Bethpage CDP, TN",0.0,0.051,0.0,0.0,0.07,0.000000


In [60]:
data = data.set_index('Industry').transpose()

In [61]:
data.head()

Industry,Construction (23),Construction (23),Construction (23),Construction (23),Construction (23),Construction (23),Construction (23),Construction (23),Construction (23),Construction (23),Construction (23),Construction (23),Construction (23),Construction (23),Construction (23),Construction (23),Construction (23),Construction (23),Construction (23),Construction (23),Construction (23),Construction (23),Construction (23),Construction (23),Construction (23),Construction (23),Construction (23),Construction (23),Construction (23),Construction (23),Construction (23),Construction (23),Construction (23),Construction (23),Construction (23),Construction (23),Construction (23),Construction (23),Construction (23),Construction (23),Construction (23),Construction (23),Construction (23),Construction (23),Construction (23),Construction (23),Construction (23),Construction (23),Construction (23),Construction (23),Construction (23),Construction (23),Construction (23),Construction (23),Construction (23),Construction (23),Construction (23),Construction (23),Construction (23),Construction (23),Construction (23),Construction (23),Construction (23),Construction (23),Construction (23),Construction (23),Construction (23),Construction (23),Construction (23),Construction (23),Construction (23),Construction (23),Construction (23),Construction (23),Construction (23),Construction (23),Construction (23),Construction (23),Construction (23),Construction (23),Construction (23),Construction (23),Construction (23),Construction (23),Construction (23),Construction (23),Construction (23),Education & Health Services,Education & Health Services,Education & Health Services,Education & Health Services,Education & Health Services,Education & Health Services,Education & Health Services,Education & Health Services,Education & Health Services,Education & Health Services,Education & Health Services,Education & Health Services,Education & Health Services,Education & Health Services,Education & Health Services,Education & Health Services,Education & Health Services,Education & Health Services,Education & Health Services,Education & Health Services,Education & Health Services,Education & Health Services,Education & Health Services,Education & Health Services,Education & Health Services,Education & Health Services,Education & Health Services,Education & Health Services,Education & Health Services,Education & Health Services,Education & Health Services,Education & Health Services,Education & Health Services,Education & Health Services,Education & Health Services,Education & Health Services,Education & Health Services,Education & Health Services,Education & Health Services,Education & Health Services,Education & Health Services,Education & Health Services,Education & Health Services,Education & Health Services,Education & Health Services,Education & Health Services,Education & Health Services,Education & Health Services,Education & Health Services,Education & Health Services,Education & Health Services,Education & Health Services,Education & Health Services,Education & Health Services,Education & Health Services,Education & Health Services,Education & Health Services,Education & Health Services,Education & Health Services,Education & Health Services,Education & Health Services,Education & Health Services,Education & Health Services,Education & Health Services,Education & Health Services,Education & Health Services,Education & Health Services,Education & Health Services,Education & Health Services,Education & Health Services,Education & Health Services,Education & Health Services,Education & Health Services,Education & Health Services,Education & Health Services,Education & Health Services,Education & Health Services,Education & Health Services,Education & Health Services,Education & Health Services,Education & Health Services,Education & Health Services,Education & Health Services,Education & Health Services,Education & Health Services,Education & Health Services,Education & Healt

In [62]:
#rename industries
data = data.rename(columns = {'Construction (23)': 'Construction', 'Information (51)': 'Information', 'Manufacturing (31)': 'Manufacturing', 
                              'Public Administration (92)': 'Public Administration', 'Total - All Industries': 'Total'}).transpose()

In [63]:
data = data.reset_index(drop = False)

In [64]:
data.head()

Value,Industry,NAME,3 Year Forecast Growth Demand,3 Year Forecast Growth Rate,Avg Ann Wages per Worker,Empl (4Q Mov Avg),LQ (4Q Mov Avg),Percent of Total
0,Construction,"Adams city, TN",0.0,0.032,0.0,0.0,0.08,0.0
1,Construction,"Ashland City town, TN",4.0,0.025,56502.0,182.0,0.61,3.63709
2,Construction,"Belle Meade city, TN",1.0,0.023,75583.0,27.0,0.34,2.047005
3,Construction,"Berry Hill city, TN",23.0,0.023,75980.0,1005.0,0.92,5.490003
4,Construction,"Bethpage CDP, TN",0.0,0.051,0.0,0.0,0.07,0.0


In [65]:
data = data.set_index('NAME').transpose()

In [66]:
data = data.rename(columns = {'Adams city, TN': 'Adams', 'Ashland City town, TN': 'Ashland City', 'Belle Meade city, TN': 'Belle Meade',
                                    'Berry Hill city, TN': 'Berry Hill', 'Brentwood city, TN': 'Brentwood', 'Burns town, TN': 'Burns',
                                    'Cedar Hill city, TN': 'Cedar Hill', 'Charlotte town, TN': 'Charlotte',
                                    'Cheatham County, Tennessee': 'Cheatham County', 'Clarksville city, TN': 'Clarksville',
                                    'Columbia city, TN': 'Columbia', 'Coopertown town, TN': 'Coopertown',
                                    'Cross Plains city, TN': 'Cross Plains', 'Cumberland City town, TN': 'Cumberland City',
                                    'Davidson County, Tennessee': 'Davidson County', 'Dickson city, TN': 'Dickson',
                                    'Dickson County, Tennessee': 'Dickson County', 'Dover city, TN': 'Dover',
                                    'Eagleville city, TN': 'Eagleville', 'Erin city, TN': 'Erin', 'Fairview city, TN': 'Fairview',
                                    'Forest Hills city, TN': 'Forest Hills', 'Franklin city, TN': 'Franklin', 'Gallatin city, TN': 'Gallatin',
                                    'Goodlettsville city, TN': 'Goodlettsville', 'Greenbrier town, TN': 'Greenbrier',
                                    'Hartsville/Trousdale County, TN': 'Hartsville/Trousdale', 'Hendersonville city, TN': 'Hendersonville',
                                    'Houston County, Tennessee': 'Houston County', 'Humphreys County, Tennessee': 'Humphreys County', 
                                    'Kingston Springs town, TN': 'Kingston Springs', 'La Vergne city, TN': 'La Vergne',
                                    'Lebanon city, TN': 'Lebanon', 'McEwen city, TN': 'McEwen','Maury County, Tennessee': 'Maury County',
                                    'Millersville city, TN': 'Millersville', 'Mitchellville city, TN': 'Mitchellville',
                                    'Montgomery County, TN': 'Montgomery County', 'Mount Juliet city, TN': 'Mount Juliet',
                                    'Mount Pleasant city, TN': 'Mount Pleasant', 'Murfreesboro city, TN': 'Murfreesboro',
                                    'Nashville-Davidson metropolitan government (balance), TN': 'Nashville', 
                                    'New Johnsonville city, TN': 'New Johnsonville', 'Nolensville town, TN': 'Nolensville',
                                    'Oak Hill city, TN': 'Oak Hill', 'Pegram town, TN': 'Pegram', 'Pleasant View city, TN': 'Pleasant View',
                                    'Portland city, TN': 'Portland', 'Ridgetop city, TN': 'Ridgetop','Robertson County, Tennessee': 'Robertson County',
                                    'Rutherford County, TN': 'Rutherford County', 'Slayden town, TN': 'Slayden', 'Smyrna town, TN': 'Smyrna',
                                    'Spring Hill city, TN': 'Spring Hill', 'Springfield city, TN': 'Springfield',
                                    'Stewart County, Tennessee': 'Stewart County', 'Sumner County, Tennessee': 'Sumner County',
                                    'Tennessee Ridge town, TN': 'Tennessee Ridge', "Thompson's Station town, TN": "Thompson's Station",
                                    'Trousdale County, Tennessee': 'Trousdale County', 'Vanleer town, TN': 'Vanleer', 'Watertown city, TN': 'Watertown',
                                    'Waverly city, TN': 'Waverly', 'Westmoreland town, TN': 'Westmoreland', 'White Bluff town, TN': 'White Bluff',
                                    'White House city, TN': 'White House', 'Williamson County, Tennessee': 'Williamson County',
                                    'Wilson County, Tennessee': 'Wilson County'})
data = data.transpose()
data = data.reset_index()

In [67]:
data.head()

Value,NAME,Industry,3 Year Forecast Growth Demand,3 Year Forecast Growth Rate,Avg Ann Wages per Worker,Empl (4Q Mov Avg),LQ (4Q Mov Avg),Percent of Total
0,Adams,Construction,0.0,0.032,0.0,0.0,0.08,0.0
1,Ashland City,Construction,4.0,0.025,56502.0,182.0,0.61,3.63709
2,Belle Meade,Construction,1.0,0.023,75583.0,27.0,0.34,2.047005
3,Berry Hill,Construction,23.0,0.023,75980.0,1005.0,0.92,5.490003
4,"Bethpage CDP, TN",Construction,0.0,0.051,0.0,0.0,0.07,0.0


In [68]:
data = data.rename(columns = {'3 Year Forecast Growth Demand': '3 Year Forecast Demand', 'Avg Ann Wages per Worker': 'Average Annual Wages', 
                              'Empl (4Q Mov Avg)': 'Employment', 'LQ (4Q Mov Avg)': 'Location Quotient'
                             })

In [69]:
data['3 Year Forecast Growth Rate'] = data['3 Year Forecast Growth Rate'] * 100
data = data.drop(columns = '3 Year Forecast Demand')

In [70]:
data.to_csv('../../Outputs/JOBSEQ_INDUSTRYEMPSTATSCURRENT_ALLGEOS.csv', index = False)